# Option Pricing for Commodities Using GARCH Volatility and FRED Data
This code calculates option prices and Greeks for various commodities (like gold, silver, and crude oil) using market data from Yahoo Finance, GARCH-based volatility, and risk-free rates from FRED.

In [ ]:
import QuantLib as ql
import yfinance as yf
import numpy as np
from fredapi import Fred
from arch import arch_model  # Import GARCH model from arch package

### Initialize FRED API
Replace with your FRED API key

In [ ]:
fred = Fred(api_key='c4bd69f7cda680b4ae018d6c2273f268')

## Function to Fetch Risk-Free Rate from FRED
This function retrieves the latest 10-year Treasury rate as the risk-free rate for option pricing.

In [ ]:
def get_risk_free_rate():
    treasury_rate = fred.get_series('DGS10')  # 10-Year Treasury Constant Maturity Rate
    return treasury_rate[-1] / 100  # Convert from percent to decimal

## GARCH Volatility Calculation
This function uses a GARCH(1,1) model to estimate the volatility of returns for use in option pricing.

In [ ]:
def calculate_garch_volatility(returns):
    model = arch_model(returns, vol='Garch', p=1, q=1)
    model_fit = model.fit(disp='off')
    forecast = model_fit.forecast(horizon=1)
    garch_volatility = np.sqrt(forecast.variance.values[-1, :][0])
    return garch_volatility

## Option Pricing Function
This function prices European-style options based on inputs such as the spot price, strike price, and GARCH volatility for a specified asset.

In [ ]:
def price_option(ticker, option_type=ql.Option.Call, strike_price=100, expiry_date=ql.Date(27, 12, 2024), 
                 dividend_rate=0.0):

    data = yf.Ticker(ticker)
    hist_data = data.history(period='1y')
    spot_price = hist_data['Close'][-1]
    log_returns = np.log(hist_data['Close'] / hist_data['Close'].shift(1)).dropna()
    garch_volatility = calculate_garch_volatility(log_returns) * np.sqrt(252)
    risk_free_rate = get_risk_free_rate()

    today = ql.Date().todaysDate()
    ql.Settings.instance().evaluationDate = today

    payoff = ql.PlainVanillaPayoff(option_type, strike_price)
    exercise = ql.EuropeanExercise(expiry_date)
    european_option = ql.VanillaOption(payoff, exercise)

    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    rate_handle = ql.YieldTermStructureHandle(ql.FlatForward(today, risk_free_rate, ql.Actual365Fixed()))
    dividend_handle = ql.YieldTermStructureHandle(ql.FlatForward(today, dividend_rate, ql.Actual365Fixed()))
    volatility_handle = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), garch_volatility, ql.Actual365Fixed()))

    bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_handle, rate_handle, volatility_handle)
    engine = ql.AnalyticEuropeanEngine(bsm_process)
    european_option.setPricingEngine(engine)

    option_price = european_option.NPV()
    delta = european_option.delta()
    gamma = european_option.gamma()
    vega = european_option.vega()
    theta = european_option.theta()
    rho = european_option.rho()

    print(f'\nOption Pricing for {ticker}')
    print(f'Spot Price: {spot_price:.2f}')
    print(f'GARCH Volatility: {garch_volatility:.4f}')
    print(f'Risk-Free Rate (from FRED): {risk_free_rate:.4f}')
    print(f'Option Price: {option_price:.2f}')
    print(f'Delta: {delta:.4f}')
    print(f'Gamma: {gamma:.4f}')
    print(f'Vega: {vega:.4f}')
    print(f'Theta: {theta:.4f}')
    print(f'Rho: {rho:.4f}')

## Example Usage for Commodities and Precious Metals
We can calculate the option price and Greeks for various commodities such as gold, silver, and crude oil.

In [ ]:
tickers = ['GC=F', 'SI=F', 'CL=F']  # Gold, Silver, Crude Oil futures tickers on Yahoo Finance

for ticker in tickers:
    price_option(ticker, strike_price=100, expiry_date=ql.Date(27, 12, 2024))